In [46]:
import pandas as pd
import numpy as np

In [47]:
# Valid until 05 march 2020

In [48]:
df1 = pd.read_csv('covid_mex_20200401.csv')

In [49]:
df2 = pd.read_csv('../data/mexico_covid_19_data/csv/raw_covid_mex_20200402.csv')
df3 = pd.read_csv('../data/mexico_covid_19_data/csv/raw_covid_mex_20200403.csv')
df4 = pd.read_csv('../data/mexico_covid_19_data/csv/raw_covid_mex_20200404.csv')
df5 = pd.read_csv('../data/mexico_covid_19_data/csv/raw_covid_mex_20200405.csv')

In [50]:
my_dfs = [df1, df2, df3, df4, df5]

for df in my_dfs:
    for column in df.columns:
        if 'Case_ID' in column:
            df.drop(column, axis = 1, inplace=True)

In [51]:
res2 = df1.combine_first(df2).reset_index()
res2.index = np.arange(1, (len(res2)+1))
res2 = res2.reset_index()
res2 = res2.rename(columns={'level_0':'Case_ID'})
res2 = res2.drop(['index'], axis=1)
res2.to_csv('covid_mex_20200402.csv', index=False)

In [53]:
res3 = df2.combine_first(df3).reset_index()
res3.index = np.arange(1, (len(res3)+1))
res3 = res3.reset_index()
res3 = res3.rename(columns={'level_0':'Case_ID'})
res3 = res3.drop(['index'], axis=1)
res3.to_csv('covid_mex_20200403.csv', index=False)

In [25]:
res4 = df3.combine_first(df3).reset_index()
res4.index = np.arange(1, (len(res4)+1))
res4 = res4.reset_index()
res4 = res4.rename(columns={'level_0':'Case_ID'})
res4 = res4.drop(['index'], axis=1)
res4.to_csv('covid_mex_20200404.csv', index=False)

In [26]:
res3 = df2.combine_first(df3).reset_index()
res3.index = np.arange(1, (len(res3)+1))
res3 = res3.reset_index()
res3 = res3.rename(columns={'level_0':'Case_ID'})
res3 = res3.drop(['index'], axis=1)
res3.to_csv('covid_mex_20200403.csv', index=False)